In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install geopandas

In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches
import geopandas as gpd
from datetime import datetime

In [ ]:
block_name = 'Mohanpur'
# block_name = 'Pindwara'
shp_path =  f'/content/drive/MyDrive/block_shape_files/{block_name}.shp'
gdf = gpd.read_file(shp_path)
# print(gdf.head(10))
filepathPrefix = f'/content/drive/MyDrive/drought_modis_mask/{block_name}_'

vcibins = [-1.0, 40.0, 60.0, 100.0]
vcilabels = ['poor', 'fair', 'good']

maibins = [-1.0, 25.0, 50.0, 100.0]
mailabels = ['poor', 'fair', 'good']

casbins = [-1.0, 33.3, 50.0, 100.0]
caslabels = ['poor', 'fair', 'good']

spibins = [-1000.0, -2.0, -1.5, -1.0, 0.0, 1.0, 1.5, 2.0, 1000.0 ]
spilabels = ['extremelyDry', 'severelyDry', 'moderatelyDry', 'mildlyDry', 'mildlyWet', 'moderatelyWet', 'severelyWet', 'extremelyWet' ]


for year in range(2003,2023):
  finpath = filepathPrefix + str(year) + ".csv"
  df = pd.read_csv(finpath)

  ##Meterological Drought START
  mDrought=0
  mDroughtCols = [col for col in df.columns if 'meteorological_drought' in col]

  df[f'mDrought_{year}'] = df[mDroughtCols].sum(axis=1)

  ##DrySPells START
  drySpells=0
  drySpellsCols = [col for col in df.columns if 'dryspell' in col]
  df[f'drySpells_{year}'] = df[drySpellsCols].sum(axis=1)

  ##Cropping Area Sown in Kharif
  df[f'cas_{year}_mode']=0
  for index, row in df.iterrows():
    val = df.at[index,f'percent_of_area_cropped_kharif_{year}']
    if(val<=33.3):
      df.at[index ,f'cas_{year}_mode']=3
    elif (val<=50.0):
      df.at[index ,f'cas_{year}_mode']=2
    else:
      df.at[index ,f'cas_{year}_mode']=1

  ##VCI START
  vciCols = [col for col in df.columns if 'vci' in col]
  for l in vcilabels:
    df[f'vci_{year}_{l}']=0

  for index, row in df.iterrows():
      for col in vciCols:
          vci_value = row[col]
          category = pd.cut([vci_value], bins=vcibins, labels=vcilabels)[0]
          df.at[index, f'vci_{year}_{category}'] += 1

  df[f'vci_{year}_mode']=0
  for index, row in df.iterrows():
    templ=[]
    i=1
    for l in vcilabels[::-1]:
      templ.append([df.at[index , f'vci_{year}_{l}'],i])
      i+=1
    templ.sort()
    df.at[index ,f'vci_{year}_mode']=templ[-1][1]


  ##MAI START
  maiCols = [col for col in df.columns if 'mai' in col]
  for l in mailabels:
    df[f'mai_{year}_{l}']=0
  for index, row in df.iterrows():
      for col in maiCols:
          mai_value = row[col]
          category = pd.cut([mai_value], bins=maibins, labels=mailabels)[0]
          df.at[index, f'mai_{year}_{category}'] += 1

  df[f'mai_{year}_mode']=0
  for index, row in df.iterrows():
    templ=[]
    i=1
    for l in mailabels[::-1]:
      # print(l)
      templ.append([df.at[index , f'mai_{year}_{l}'],i])
      i+=1
    templ.sort()
    df.at[index ,f'mai_{year}_mode']=templ[-1][1]



  ##SPI START
  spiCols = [col for col in df.columns if 'spi' in col]
  for l in spilabels:
    df[f'spi_{year}_{l}']=0
  for index, row in df.iterrows():
      for col in spiCols:
          spi_value = row[col]
          category = pd.cut([spi_value], bins=spibins, labels=spilabels)[0]
          df.at[index, f'spi_{year}_{category}'] += 1

# ['extremelyDry', 'severelyDry', 'moderatelyDry', 'mildlyDry', 'mildlyWet', 'moderatelyWet', 'severelyWet', 'extremelyWet' ]
  df[f'spi_{year}_mode']=0
  for index, row in df.iterrows():
    templ=[]
    i=1
    for l in spilabels[::-1]:
      templ.append([df.at[index , f'spi_{year}_{l}'],i])
      i+=1
    templ.sort()
    df.at[index ,f'spi_{year}_mode']=templ[-1][1]


  # print(len(mDroughtCols))
  # print(len(drySpellsCols))
  # print(len(vciCols))
  # print(len(maiCols))
  # print(len(spiCols))
  gdf = pd.merge(gdf,df, on='uid', how='left')

# print(gdf.columns.tolist)

# print(gdf[['vci_2003_good','vci_2003_fair', 'vci_2003_poor', 'mDrought_2003', 'drySpells_2003',
#            'mai_2003_no', 'mai_2003_mild', 'mai_2003_moderate', 'mai_2003_severe',
#            'spi_2003_extremelyDry', 'spi_2003_severelyDry', 'spi_2003_moderatelyDry', 'spi_2003_mildlyDry',
#            'spi_2003_mildlyWet', 'spi_2003_moderatelyWet', 'spi_2003_severelyWet', 'spi_2003_extremelyWet'
#            ]])


<ipython-input-4-513b022fdfaa>:119: FutureWarning: Passing 'suffixes' which cause duplicate columns {'area_in_ha_x', 'id_x'} in the result is deprecated and will raise a MergeError in a future version.
  gdf = pd.merge(gdf,df, on='uid', how='left')
<ipython-input-4-513b022fdfaa>:119: FutureWarning: Passing 'suffixes' which cause duplicate columns {'system:index_x', 'kharif_croppable_sqkm_x', '.geo_x'} in the result is deprecated and will raise a MergeError in a future version.
  gdf = pd.merge(gdf,df, on='uid', how='left')
<ipython-input-4-513b022fdfaa>:119: FutureWarning: Passing 'suffixes' which cause duplicate columns {'area_in_ha_x', 'id_x'} in the result is deprecated and will raise a MergeError in a future version.
  gdf = pd.merge(gdf,df, on='uid', how='left')
<ipython-input-4-513b022fdfaa>:119: FutureWarning: Passing 'suffixes' which cause duplicate columns {'system:index_x', 'kharif_croppable_sqkm_x', '.geo_x'} in the result is deprecated and will raise a MergeError in a futur

In [ ]:
print(gdf.head(10))
print(gdf.columns.tolist())

         uid  area_in_ha_x                  id_x  \
0  12_184719   1760.596962  00000000000000000000   
1  12_185280   1193.353512  00000000000000000001   
2  12_312390    709.236192  0000000000000000000c   
3  12_313101   2654.015081  0000000000000000000d   
4  12_314973   2463.552119  00000000000000000019   
5  12_315309   2503.568658  0000000000000000001a   
6  12_317748   1491.961687  0000000000000000001d   
7  12_321631   1001.780695  00000000000000000020   
8  12_322127   3455.757847  00000000000000000021   
9  12_322417   1389.542389  00000000000000000022   

                                            geometry        system:index_x  \
0  POLYGON ((84.93810 24.57615, 84.93810 24.57642...  00000000000000000000   
1  POLYGON ((84.93514 24.55702, 84.93487 24.55702...  00000000000000000001   
2  POLYGON ((85.21514 24.52711, 85.21514 24.52818...  00000000000000000002   
3  POLYGON ((85.20787 24.47105, 85.20787 24.47186...  00000000000000000003   
4  POLYGON ((85.13510 24.60634, 85.13

**STEP2**

In [ ]:
map = {
    'severe': 3,
    'moderate': 2,
    'mild': 1
    }

def getWeekVector(fin_df,wkCt,year):
  # print("after ", new_df.head(10))
  for index, row in fin_df.iterrows():
    ds = row[f'dryspell_{year}_week_{wkCt}']
    rfdev = row[f'monthly_rainfall_deviation_{year}_week_{wkCt}']
    spi = row[f'spi_{year}_week_{wkCt}']
    vci = row[f'vci_{year}_week_{wkCt}']
    mai = row[f'mai_{year}_week_{wkCt}']
    cas = row[f'kharif_cropped_sqkm_{year}']
    # print(row,"==>",ds,rfdev,spi,vci,mai,cas)

    vci_class = 'mild'
    if(60<vci and vci<=100):
      vci_class='severe'
    elif(40<vci and vci<=60):
      vci_class='moderate'
    else:
      vci_class='mild'

    mai_class = 'mild'
    if(mai<=25):
      mai_class = 'severe'
    elif(mai<=50):
      mai_class = 'moderate'
    else:
      mai_class = 'mild'

    cas_class = 'mild'
    if(cas<=33.3):
      cas_class = 'severe'
    elif(cas<=50):
      cas_class = 'moderate'
    else:
      cas_class = 'mild'

    mD = 0
    if(ds==1 or rfdev=='scanty' or spi<-1.5):
      mD = 1
    if(mD==1):
      if(vci_class=='severe' and mai_class=='severe' and cas_class=='severe'):
        if(ds==1):
          fin_df.at[index,f'severe_drought_path1_{year}']+=1
        elif(rfdev=='scanty'):
          fin_df.at[index,f'severe_drought_path2_{year}']+=1
        else:
          fin_df.at[index,f'severe_drought_path3_{year}']+=1
      elif(vci_class=='mild' and mai_class=='severe' and cas_class=='severe'):
        if(ds==1):
          fin_df.at[index,f'moderate_drought_path1_{year}']+=1
        elif(rfdev=='scanty'):
          fin_df.at[index,f'moderate_drought_path2_{year}']+=1
        else:
          fin_df.at[index,f'moderate_drought_path3_{year}']+=1
      elif(vci_class=='moderate' and mai_class=='severe' and cas_class=='severe'):
        if(ds==1):
          fin_df.at[index,f'moderate_drought_path4_{year}']+=1
        elif(rfdev=='scanty'):
          fin_df.at[index,f'moderate_drought_path5_{year}']+=1
        else:
          fin_df.at[index,f'moderate_drought_path6_{year}']+=1
      elif(vci_class=='severe' and mai_class=='mild' and cas_class=='severe'):
        if(ds==1):
          fin_df.at[index,f'moderate_drought_path7_{year}']+=1
        elif(rfdev=='scanty'):
          fin_df.at[index,f'moderate_drought_path8_{year}']+=1
        else:
          fin_df.at[index,f'moderate_drought_path9_{year}']+=1
      elif(vci_class=='severe' and mai_class=='moderate' and cas_class=='severe'):
        if(ds==1):
          fin_df.at[index,f'moderate_drought_path10_{year}']+=1
        elif(rfdev=='scanty'):
          fin_df.at[index,f'moderate_drought_path11_{year}']+=1
        else:
          fin_df.at[index,f'moderate_drought_path12_{year}']+=1
      elif(vci_class=='severe' and mai_class=='severe' and cas_class=='mild'):
        if(ds==1):
          fin_df.at[index,f'moderate_drought_path13_{year}']+=1
        elif(rfdev=='scanty'):
          fin_df.at[index,f'moderate_drought_path14_{year}']+=1
        else:
          fin_df.at[index,f'moderate_drought_path15_{year}']+=1
      elif(vci_class=='severe' and mai_class=='severe' and cas_class=='moderate'):
        if(ds==1):
          fin_df.at[index,f'moderate_drought_path16_{year}']+=1
        elif(rfdev=='scanty'):
          fin_df.at[index,f'moderate_drought_path17_{year}']+=1
        else:
          fin_df.at[index,f'moderate_drought_path18_{year}']+=1
      else:
        if(ds==1):
          fin_df.at[index,f'mild_drought_dryspell_score_{year}']+=1
        elif(rfdev=='scanty'):
          fin_df.at[index,f'mild_drought_rainfall_deviation_score_{year}']+=1
        elif(spi<-1.5):
          fin_df.at[index,f'mild_drought_spi_score_{year}']+=1
        fin_df.at[index,f'mild_drought_vci_score_{year}']+=map[vci_class]
        fin_df.at[index,f'mild_drought_mai_score_{year}']+=map[mai_class]
        fin_df.at[index,f'mild_drought_cropping_area_sown_score_{year}']+=map[cas_class]
"""
VCI Value (%) Vegetation Condition
60-100 Good
40-60 Fair
0-40 Poor


MAI (%) Agricultural Drought Class
51-100 Mild drought
26-50 Moderate drought
0-25 Severe drought

Area Sown(%) Drought Condition
0-33.3 Severe drought
33.3-50 Moderate drought
50-100 Mild drought

"""


'\nVCI Value (%) Vegetation Condition\n60-100 Good\n40-60 Fair\n0-40 Poor\n\n\nMAI (%) Agricultural Drought Class\n51-100 Mild drought\n26-50 Moderate drought\n0-25 Severe drought\n\nArea Sown(%) Drought Condition\n0-33.3 Severe drought\n33.3-50 Moderate drought\n50-100 Mild drought\n\n'

In [ ]:
labels = ['dryspell','mai','meteorological_drought', 'monthly_rainfall_deviation', 'spi', 'rfdev_class', 'vci']
singleLabels = ['monsoon_onset', 'total_weeks', 'kharif_cropped_sqkm']

create_cols = ['frequency_of_no_drought', 'frequency_of_mild_drought', 'frequency_of_moderate_drought', 'frequency_of_severe_drought',
               'intensity_of_no_drought', 'intensity_of_mild_drought', 'intensity_of_moderate_drought', 'intensity_of_severe_drought',
               'severe_drought_path1', 'severe_drought_path2', 'severe_drought_path3',
               'moderate_drought_path1', 'moderate_drought_path2', 'moderate_drought_path3', 'moderate_drought_path4', 'moderate_drought_path5',
               'moderate_drought_path6', 'moderate_drought_path7', 'moderate_drought_path8', 'moderate_drought_path9', 'moderate_drought_path10',
               'moderate_drought_path11', 'moderate_drought_path12', 'moderate_drought_path13', 'moderate_drought_path14', 'moderate_drought_path15',
               'moderate_drought_path16', 'moderate_drought_path17', 'moderate_drought_path18',
               'mild_drought_dryspell_score', 'mild_drought_rainfall_deviation_score', 'mild_drought_spi_score',
               'mild_drought_vci_score', 'mild_drought_mai_score', 'mild_drought_cropping_area_sown_score'
               ]

fin_df = pd.DataFrame()
fin_df['uid'] = gdf['uid']

for year in range(2003,2023):
  for col in create_cols:
    fin_df[f'{col}_{year}'] = 0
  fin_df[f'frequency_of_no_drought_{year}'] = gdf[f'freq_of_drought_{year}_at_threshold_0']
  fin_df[f'frequency_of_mild_drought_{year}'] = gdf[f'freq_of_drought_{year}_at_threshold_1']
  fin_df[f'frequency_of_moderate_drought_{year}'] = gdf[f'freq_of_drought_{year}_at_threshold_2']
  fin_df[f'frequency_of_severe_drought_{year}'] = gdf[f'freq_of_drought_{year}_at_threshold_3']
  fin_df[f'intensity_of_no_drought_{year}'] = gdf[f'intensity_of_drought_{year}_at_threshold_0']
  fin_df[f'intensity_of_mild_drought_{year}'] = gdf[f'intensity_of_drought_{year}_at_threshold_1']
  fin_df[f'intensity_of_moderate_drought_{year}'] = gdf[f'intensity_of_drought_{year}_at_threshold_2']
  fin_df[f'intensity_of_severe_drought_{year}'] = gdf[f'intensity_of_drought_{year}_at_threshold_3']

  prefixName = f"weekly_label_{year}"
  # print(prefixName)
  dates = [col[13:] for col in gdf.columns if (prefixName in col)]
  # print(dates)
  date_objects = [[datetime.strptime(date, '%Y-%m-%d'),date] for date in dates]
  # Sort datetime objects
  sorted_dates = sorted(date_objects)
  sorted_dates_strings = [date[1] for date in sorted_dates]
  # print(len(sorted_dates_strings))
  # print( gdf[f'total_weeks_{year}'])

  # new_df = pd.DataFrame()
  # new_df['uid'] = gdf['uid']

  new_cols = []
  for label in singleLabels:
    colname = f'{label}_{year}'
    fin_df[colname]=gdf[colname]
    new_cols.append(colname)

  for label in labels:
    for i in range(len(sorted_dates_strings)):
      s=sorted_dates_strings[i]
      col = f'{label}_{s}'
      colnew = f'{label}_{year}_week_{i+1}'
      fin_df[colnew] = gdf[col]
      new_cols.append(colnew)
  # for col in new_df.columns:
    # print(col)

  for i in range(len(sorted_dates_strings)):
    weekCt = i+1
    # new_df[f'vector_{year}_week_{weekCt}'] = np.nan
    # print("before ", new_df.head(10))
    getWeekVector(fin_df,weekCt,year)
    # print(new_df[f'vector_{year}_week_{weekCt}'])

  fin_df.drop(new_cols,axis=1,inplace=True)
  fin_df[f'mild_drought_vci_score_{year}']/=6
  fin_df[f'mild_drought_mai_score_{year}']/=6
  fin_df[f'mild_drought_cropping_area_sown_score_{year}']/=6


<ipython-input-7-aecc8a14437e>:55: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fin_df[colnew] = gdf[col]
<ipython-input-7-aecc8a14437e>:55: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fin_df[colnew] = gdf[col]
<ipython-input-7-aecc8a14437e>:55: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fin_df

In [ ]:
print(fin_df.head(10))
print(fin_df.columns.tolist())

         uid  frequency_of_no_drought_2003  frequency_of_mild_drought_2003  \
0  12_184719                            23                              22   
1  12_185280                            23                              22   
2  12_312390                            23                              22   
3  12_313101                            23                              22   
4  12_314973                            23                              22   
5  12_315309                            23                              22   
6  12_317748                            23                              23   
7  12_321631                            23                              23   
8  12_322127                            23                              23   
9  12_322417                            23                              23   

   frequency_of_moderate_drought_2003  frequency_of_severe_drought_2003  \
0                                   0                             

**STEP3**

In [ ]:
def count1(row, year):
    index = row.name
    paths = [
        [ 'severe_drought_path1', 0 ],
        [ 'severe_drought_path2', 0 ],
        [ 'severe_drought_path3', 0 ],
        [ 'moderate_drought_path1', 0 ],
        [ 'moderate_drought_path2', 0 ],
        [ 'moderate_drought_path3', 0 ],
        [ 'moderate_drought_path4', 0 ],
        [ 'moderate_drought_path5', 0 ],
        [ 'moderate_drought_path6', 0 ],
        [ 'moderate_drought_path7', 0 ],
        [ 'moderate_drought_path8', 0 ],
        [ 'moderate_drought_path9', 0 ],
        [ 'moderate_drought_path10', 0 ],
        [ 'moderate_drought_path11', 0 ],
        [ 'moderate_drought_path12', 0 ],
        [ 'moderate_drought_path13', 0 ],
        [ 'moderate_drought_path14', 0 ],
        [ 'moderate_drought_path15', 0 ],
        [ 'moderate_drought_path16', 0 ],
        [ 'moderate_drought_path17', 0 ],
        [ 'moderate_drought_path18', 0 ],
    ]
    for index, path in enumerate(paths):
        prefix, count = path
        col = f"{prefix}_{year}"
        count += row[col]
        paths[index][1]=count
    paths = sorted(paths, key=lambda x:x[1], reverse=True)[:3]
    output = ""
    for prefix, count in paths:
        if count==0: continue
        output = output + ' | ' + prefix + f"({count})"
    return output[2:]

def count2(row, year):
    index = row.name
    nodes = [
        ['mild_drought_dryspell_score', row[f'mild_drought_dryspell_score_{year}']],
        ['mild_drought_rainfall_deviation_score', row[f'mild_drought_rainfall_deviation_score_{year}']],
        ['mild_drought_spi_score', row[f'mild_drought_spi_score_{year}']],
        ['mild_drought_vci_score', row[f'mild_drought_vci_score_{year}']],
        ['mild_drought_mai_score', row[f'mild_drought_mai_score_{year}']],
        ['mild_drought_cropping_area_sown_score', row[f'mild_drought_cropping_area_sown_score_{year}']],
    ]
    nodes = sorted(nodes, key=lambda x:x[1], reverse=True)[:3]
    output = ""
    for prefix, count in nodes:
        if count==0: continue
        output = output + ' | ' + prefix + f"({count})"
    return output[2:]


data = fin_df
for year in range(2003, 2023):
    data['severe_moderate_drought_causality_'+str(year)] = data.apply(lambda row : count1(row, year), axis=1)
    data['mild_drought_causality_'+str(year)] = data.apply(lambda row : count2(row, year), axis=1)

In [ ]:
print(data.columns.tolist())
print(data.head(10))

['uid', 'frequency_of_no_drought_2003', 'frequency_of_mild_drought_2003', 'frequency_of_moderate_drought_2003', 'frequency_of_severe_drought_2003', 'intensity_of_no_drought_2003', 'intensity_of_mild_drought_2003', 'intensity_of_moderate_drought_2003', 'intensity_of_severe_drought_2003', 'severe_drought_path1_2003', 'severe_drought_path2_2003', 'severe_drought_path3_2003', 'moderate_drought_path1_2003', 'moderate_drought_path2_2003', 'moderate_drought_path3_2003', 'moderate_drought_path4_2003', 'moderate_drought_path5_2003', 'moderate_drought_path6_2003', 'moderate_drought_path7_2003', 'moderate_drought_path8_2003', 'moderate_drought_path9_2003', 'moderate_drought_path10_2003', 'moderate_drought_path11_2003', 'moderate_drought_path12_2003', 'moderate_drought_path13_2003', 'moderate_drought_path14_2003', 'moderate_drought_path15_2003', 'moderate_drought_path16_2003', 'moderate_drought_path17_2003', 'moderate_drought_path18_2003', 'mild_drought_dryspell_score_2003', 'mild_drought_rainfall

In [ ]:
output_path = f'/content/drive/MyDrive/drought_modis_mask/{block_name}_drought_causality.csv'
data.to_csv(output_path, index=False)